In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import warnings
from scipy.stats import poisson
from tqdm import tqdm 
warnings.filterwarnings('ignore')


In [48]:
years = ['2018', '2019', '2020', '2021', '2022', '2023']
file_paths = [
    'data/VFT_Count_Zip_2018.csv',
    'data/VFT_Count_Zip_2019.csv',
    'data/VFT_Count_Zip_2020.csv',
    'data/VFT_Count_Zip_2021.csv',
    'data/VFT_Count_Zip_2022.csv',
    'data/VFT_Count_Zip_2023.csv'
]

# Load files with column renaming for VFT_Count_Zip_2023.csv
dfs = []
for file in file_paths:
    if '2023' in file:
        df = pd.read_csv(file).rename(columns={'ZIP Code': 'Zip Code'})
    else:
        df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all data into one DataFrame
vft_data = pd.concat(dfs)

# Replacement dictionary
replacements = {
    '12/31/2023': '2023',
    '12/31/2022': '2022',
    '1/1/2022': '2021',
    '1/1/2021': '2020',
    '1/1/2020': '2019',
    '10/1/2018': '2018'
}

# Replace the values in the DataFrame
vft_data.replace(replacements, inplace=True)


In [49]:
stations_df = pd.read_csv('data/filtered_alt_fuel_stations.csv')

In [50]:

data = vft_data


In [51]:
merged_df = pd.merge(stations_df, vft_data, left_on='zip', right_on='Zip Code', how='inner')


In [52]:
merged_df

,access_code,access_days_time,access_detail_code,cards_accepted,date_last_confirmed,expected_date,fuel_type_code,groups_with_access_code,id,maximum_vehicle_class,...,federal_agency.id,federal_agency.code,federal_agency.name,Date,Zip Code,Model Year,Fuel,Make,Duty,Vehicles
0,public,24 hours daily,NaN,NaN,2024-08-15,NaN,ELEC,Public,6355,NaN,...,NaN,NaN,NaN,2018,92037,2006,Diesel and Diesel Hybrid,OTHER/UNK,Heavy,9
1,public,24 hours daily,NaN,NaN,2024-08-15,NaN,ELEC,Public,6355,NaN,...,NaN,NaN,NaN,2018,92037,2006,Diesel and Diesel Hybrid,OTHER/UNK,Light,15
2,public,24 hours daily,NaN,NaN,2024-08-15,NaN,ELEC,Public,6355,NaN,...,NaN,NaN,NaN,2018,92037,2006,Battery Electric,OTHER/UNK,Light,4
3,public,24 hours daily,NaN,NaN,2024-08-15,NaN,ELEC,Public,6355,NaN,...,NaN,NaN,NaN,2018,92037,2006,Flex-Fuel,OTHER/UNK,Light,13
4,public,24 hours daily,NaN,NaN,2024-08-15,NaN,ELEC,Public,6355,NaN,...,NaN,NaN,NaN,2018,92037,2006,Gasoline,ACURA,Light,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3052607,public,24 hours daily,NaN,NaN,2024-12-03,NaN,ELEC,Public,353883,NaN,...,NaN,NaN,NaN,2022,91931,<2009,Unk,OTHER/UNK,Unk,14
3052608,public,24 hours daily,NaN,NaN,2024-12-03,NaN,ELEC,Public,353883,NaN,...,NaN,NaN,NaN,2022,91931,<2009,Diesel and Diesel Hybrid,OTHER/UNK,Heavy,4
3052609,public,24 hours daily,NaN,NaN,2024-12-03,NaN,ELEC,Public,353883,NaN,...,NaN,NaN,NaN,2022,91931,<2009,Diesel and Diesel Hybrid,OTHER/UNK,Light,12
3052610,public,24 hours daily,NaN,NaN,2024-12-03,NaN,ELEC,Public,353883,NaN,...,NaN,NaN,NaN,2022,91931,<2009,Flex-Fuel,OTHER/UNK,Light,11


In [53]:
merged_df = merged_df.dropna(axis=1, how='any')

In [54]:
data = merged_df

# Step 2: Ensure unique entries for each 'Zip Code' and 'Date' combination
aggregated_data = data.groupby(['Zip Code', 'Date'], as_index=False)['Vehicles'].sum()

# Step 3: Pivot the dataframe to have 'Zip Code' as rows and 'Date' as columns
pivot_data = aggregated_data.pivot(index='Zip Code', columns='Date', values='Vehicles')

# Step 4: Melt the dataframe back to long format
melted_data = pivot_data.reset_index().melt(
    id_vars=['Zip Code'], 
    var_name='date', 
    value_name='counts'
)

# Step 5: Fit a Poisson distribution for each 'Zip Code'
poisson_params = {}
for zip_code in melted_data['Zip Code'].unique():
    counts = melted_data[melted_data['Zip Code'] == zip_code]['counts'].dropna()
    lambda_param = np.mean(counts)  # Mean serves as the lambda parameter
    poisson_params[zip_code] = lambda_param

# Step 6: Convert Poisson parameters to a DataFrame for better visibility
poisson_params_df = pd.DataFrame.from_dict(poisson_params, orient='index', columns=['Lambda']).reset_index()
poisson_params_df.rename(columns={'index': 'Zip Code'}, inplace=True)

# Display the final Poisson parameters DataFrame
print(poisson_params_df)

# Optional: Save the results to a CSV file
output_path = 'result_data/poisson_parameters.csv' 
poisson_params_df.to_csv(output_path, index=False)


     Zip Code        Lambda
0       91901  4.696300e+04
1       91906  3.107400e+04
2       91910  1.387217e+06
3       91911  2.237094e+06
4       91913  3.995800e+04
..        ...           ...
104     92691  3.275893e+05
105     92692  4.336530e+05
106     92694  4.414980e+05
107     92697  9.170000e+02
108     95126  6.714750e+05

[109 rows x 2 columns]


In [56]:
pd.read_csv('result_data/poisson_parameters.csv')

,Zip Code,Lambda
0,91901,4.696300e+04
1,91906,3.107400e+04
2,91910,1.387217e+06
3,91911,2.237094e+06
4,91913,3.995800e+04
...,...,...
104,92691,3.275893e+05
105,92692,4.336530e+05
106,92694,4.414980e+05
107,92697,9.170000e+02


In [57]:


# Step 1: Load the data

data = merged_df

# Step 2: Ensure unique entries for each 'Zip Code' and 'Date' combination
aggregated_data = data.groupby(['Zip Code', 'Date'], as_index=False)['Vehicles'].sum()

# Step 3: Pivot the dataframe to have 'Zip Code' as rows and 'Date' as columns
pivot_data = aggregated_data.pivot(index='Zip Code', columns='Date', values='Vehicles')

# Step 4: Melt the dataframe back to long format
melted_data = pivot_data.reset_index().melt(
    id_vars=['Zip Code'], 
    var_name='date', 
    value_name='counts'
)

# Step 5: Fit a Poisson distribution for each 'Zip Code'
poisson_params = {}
for zip_code in melted_data['Zip Code'].unique():
    counts = melted_data[melted_data['Zip Code'] == zip_code]['counts'].dropna()
    lambda_param = np.mean(counts)  # Mean serves as the lambda parameter
    poisson_params[zip_code] = lambda_param

# Step 6: Monte Carlo Simulation
n_samples = 1000  # Number of simulations
samples_results = {}

for n in tqdm(range(n_samples), desc="Monte Carlo Simulation"):
    results = []
    for zip_code, lambda_param in poisson_params.items():
        # Generate a random sample from Poisson distribution
        value = poisson.rvs(mu=lambda_param, size=1)[0]
        results.append(value)
    samples_results[n] = results

# Convert simulation results to DataFrame
df_results = pd.DataFrame(samples_results).T  # Transpose so each sample is a column
df_results.columns = poisson_params.keys()  # Use Zip Codes as column names

# Step 7: Analyze results
df_results['row_sum'] = df_results.sum(axis=1)  # Total counts for each simulation
total_sum = df_results.sum(axis=0)  # Total counts for each Zip Code across simulations
row_means = df_results.mean(axis=1)  # Mean of row-wise totals

# Optional: Save results to a CSV
output_path = 'result_data/monte_carlo_results.csv'  # 替换为你的目标路径
df_results.to_csv(output_path, index=False)

# Print key results
print("Row-wise totals (sum):\n", df_results['row_sum'].head())
print("Column-wise totals (sum):\n", total_sum.head())
print("Row-wise means:\n", row_means.head())


Monte Carlo Simulation: 100%|██████████████| 1000/1000 [00:02<00:00, 446.40it/s]


Row-wise totals (sum):
 0    54329234
1    54316777
2    54321148
3    54312862
4    54325772
Name: row_sum, dtype: int64
Column-wise totals (sum):
 91901      46968122
91906      31074825
91910    1387234640
91911    2237181973
91913      39963390
dtype: int64
Row-wise means:
 0    987804.254545
1    987577.763636
2    987657.236364
3    987506.581818
4    987741.309091
dtype: float64


In [58]:
pd.read_csv('result_data/monte_carlo_results.csv')

,91901,91906,91910,91911,91913,91914,91915,91931,91932,91935,...,92675,92677,92679,92688,92691,92692,92694,92697,95126,row_sum
0,47038,30810,1388644,2235668,39838,86932,292231,592,110593,9389,...,254922,759468,27053,107707,328581,433383,442202,851,671943,54329234
1,47371,31516,1389366,2238335,39645,86904,292579,586,110389,9220,...,254629,759118,27337,107694,328201,434145,442201,879,672144,54316777
2,47413,31167,1386381,2235192,39663,86548,292500,618,110448,9274,...,254897,758287,27118,106914,327533,433748,441964,854,672585,54321148
3,47030,31122,1388304,2235339,39761,87245,292882,608,110608,9251,...,255314,759148,27426,108020,327381,433948,441256,893,671610,54312862
4,47206,31147,1386178,2237742,39605,87030,293126,618,109768,9405,...,253720,759420,27418,107369,328145,433611,442260,947,671041,54325772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,47074,31198,1385676,2237150,39867,86364,292336,592,110633,9251,...,255094,759244,27494,107562,327968,433493,440643,923,672891,54310329
996,46985,31175,1388083,2237706,40212,86155,293491,639,110997,9344,...,254715,758831,27213,107313,326725,433913,440703,965,670694,54315213
997,47138,30992,1389219,2237574,39882,86723,292789,566,111080,9345,...,255059,758829,27225,106770,328278,433755,441079,900,671550,54322370
998,46960,31081,1385517,2237867,39516,86826,292694,596,110708,9385,...,255425,759988,27237,106871,327850,433860,441974,885,671190,54328982


In [59]:
def analyze_ev_registrations(df):
    """
    Analyze EV registrations by zip code and fit Poisson distribution using mean rate
    
    Parameters:
    df (pandas.DataFrame): DataFrame with columns for zip code, year, and vehicle data
    
    Returns:
    pandas.DataFrame: Analysis results with zip codes and fitted lambda parameters
    """
    # Group by zip code and year to get counts
    ev_counts = df.groupby(['Zip Code', 'Date'])['Vehicles'].sum().reset_index()
    
    results = []
    
    # Calculate Poisson lambda (mean rate) for each zip code
    for zip_code in ev_counts['Zip Code'].unique():
        zip_data = ev_counts[ev_counts['Zip Code'] == zip_code]
        
        if len(zip_data) > 0:
            # For Poisson distribution, lambda is simply the mean rate
            lambda_param = zip_data['Vehicles'].mean()
            
            # Calculate goodness of fit using Chi-square test
            observed = zip_data['Vehicles'].values
            expected = np.full_like(observed, lambda_param)
            
            try:
                chi_square, p_value = stats.chisquare(observed, expected)
                
                results.append({
                    'zip': zip_code,
                    'lambda': lambda_param,
                    'total_evs': zip_data['Vehicles'].sum(),
                    'num_years': len(zip_data),
                    'avg_evs_per_year': lambda_param,
                    'p_value': p_value
                })
            except:
                results.append({
                    'zip': zip_code,
                    'lambda': lambda_param,
                    'total_evs': zip_data['Vehicles'].sum(),
                    'num_years': len(zip_data),
                    'avg_evs_per_year': lambda_param,
                    'p_value': None
                })
    
    return pd.DataFrame(results)

# Example usage:
results_df = analyze_ev_registrations(merged_df)


In [60]:
results_df.to_csv('result_data/result.csv', index=False)

In [61]:
def create_interactive_analysis(data_path, output_path='dashboard.html'):
    """
    Create interactive analysis and save as HTML
    
    Parameters:
    data_path: str - path to the CSV data file
    output_path: str - path where to save the HTML file
    """
    # Read the data
    df = pd.read_csv(data_path)
    
    # Create figures
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Top 10 ZIP Codes by Annual Average',
                       'Distribution of Registration Volumes',
                       'P-Value Distribution',
                       'Registration Volume vs P-Value'),
        specs=[[{"type": "bar"}, {"type": "histogram"}],
               [{"type": "histogram"}, {"type": "scatter"}]]
    )
    
    # 1. Top 10 ZIP codes bar chart
    top_10 = df.nlargest(10, 'lambda')
    fig.add_trace(
        go.Bar(
            x=top_10['zip'].astype(str),
            y=top_10['lambda'],
            name='Annual Average',
            text=top_10['lambda'].round(0),
            textposition='auto',
            hovertemplate="ZIP: %{x}<br>Annual Average: %{y:,.0f}<extra></extra>"
        ),
        row=1, col=1
    )
    
    # 2. Distribution of registration volumes
    fig.add_trace(
        go.Histogram(
            x=df['lambda'],
            name='Registration Distribution',
            nbinsx=30,
            hovertemplate="Range: %{x:,.0f}<br>Count: %{y}<extra></extra>"
        ),
        row=1, col=2
    )
    
    # 3. P-value distribution (excluding null values)
    p_values = df['p_value'].dropna()
    fig.add_trace(
        go.Histogram(
            x=p_values,
            name='P-Value Distribution',
            nbinsx=30,
            hovertemplate="P-Value: %{x}<br>Count: %{y}<extra></extra>"
        ),
        row=2, col=1
    )
    
    # 4. Scatter plot of registration volume vs p-value
    fig.add_trace(
        go.Scatter(
            x=df['lambda'],
            y=df['p_value'],
            mode='markers',
            name='Volume vs P-Value',
            text=df['zip'],  # Add ZIP codes for hover info
            marker=dict(
                size=8,
                color=df['total_evs'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Total EVs")
            ),
            hovertemplate="ZIP: %{text}<br>Annual Avg: %{x:,.0f}<br>P-Value: %{y:.3f}<extra></extra>"
        ),
        row=2, col=2
    )
    
    # Update layout with better styling
    fig.update_layout(
        height=800,
        width=1200,
        showlegend=False,
        title_text="Vehicle Registration Analysis Dashboard",
        title_x=0.5,  # Center the title
        template="plotly_white",  # Use a clean template
        font=dict(family="Arial, sans-serif", size=12),
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="ZIP Code", row=1, col=1)
    fig.update_yaxes(title_text="Annual Average Registrations", row=1, col=1)
    fig.update_xaxes(title_text="Annual Average Registrations", row=1, col=2)
    fig.update_yaxes(title_text="Count", row=1, col=2)
    fig.update_xaxes(title_text="P-Value", row=2, col=1)
    fig.update_yaxes(title_text="Count", row=2, col=1)
    fig.update_xaxes(title_text="Annual Average Registrations", row=2, col=2)
    fig.update_yaxes(title_text="P-Value", row=2, col=2)
    
    # Add better hover interactions
    fig.update_layout(hovermode='closest')
    
    # Save as interactive HTML
    fig.write_html(
        output_path,
        include_plotlyjs=True,  
        full_html=True,  
        config={
            'displayModeBar': True,  
            'scrollZoom': True,      
            'modeBarButtonsToAdd': ['drawopenpath', 'eraseshape'],  
            'toImageButtonOptions': {'height': None, 'width': None}  
        }
    )
    
    print(f"Interactive dashboard has been saved to {output_path}")
    return fig


if __name__ == "__main__":
    try:

        fig = create_interactive_analysis('result_data/result.csv', 'Interactive_output/vehicle_analysis_dashboard.html')
        

        df = pd.read_csv('result_data/result.csv')
        print("\nData Summary:")
        print("-" * 50)
        print(f"Total ZIP codes analyzed: {len(df)}")
        print(f"Average vehicles per ZIP code: {df['lambda'].mean():,.0f}")
        print(f"Maximum vehicles in a ZIP code: {df['lambda'].max():,.0f}")
        print(f"Minimum vehicles in a ZIP code: {df['lambda'].min():,.0f}")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

Interactive dashboard has been saved to Interactive_output/vehicle_analysis_dashboard.html

Data Summary:
--------------------------------------------------
Total ZIP codes analyzed: 109
Average vehicles per ZIP code: 498,319
Maximum vehicles in a ZIP code: 7,694,076
Minimum vehicles in a ZIP code: 27
